
# PINN com Keras/TensorFlow: Equação de Laplace radial (região externa)
## Alexandre Suaide

Este notebook resolve com uma **rede neural informada por física (PINN)** a equação de Laplace radial em 2D na região
**externa** de um cilindro de raio `R` com potencial fixo `V(R)=V0`.  
Precisamos definir um raio `R_ext` com potencial fixo `V(R_ext) = V_ext` como condição de contorno, já que o problema tem que ser finito.

**Equação (R ≤ r ≤ R_ext):**  
$$ \frac{1}{r} \frac{d}{dr} ( r \frac{dV}{dr} ) = 0 $$

**Condições de contorno:**  
- `V(R) = V0`  
- `V(R_ext) = V_ext`  

A solução analítica é do tipo:  
$$ V(r) = A \ln(r) +B $$


In [ ]:
# Se estiver no Google Colab, descomente a linha abaixo para instalar dependências:
# !pip install tensorflow --quiet

In [ ]:
# importa os módulos necessários
# dependendo da versão do TensoFlow, aparece um monte de warnings

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

tf.random.set_seed(1234)
np.random.seed(1234)


---
### Parâmetros físicos

In [ ]:

# condições de contorno. Define o raio o cilindro, o raio externo e os potenciais

R = 1.0
V0 = 5.0
R_ext = 5.0
V_ext = 0.0

# Amostragem dos pontos

N_interior = 2000
N_bc_R = 200
N_bc_Rext = 200

# Amostras no interior, entre o cilindro e o raio externo
r_interior = np.random.uniform(R, R_ext, (N_interior,1)).astype(np.float32)

# Valores nas Condições de contorno
r_bc_R = np.full((N_bc_R,1), R, dtype=np.float32)
r_bc_Rext = np.full((N_bc_Rext,1), R_ext, dtype=np.float32)
V_R_target = np.full((N_bc_R,1), V0, dtype=np.float32)
V_Rext_target = np.full((N_bc_Rext,1), V_ext, dtype=np.float32)


---
### constroi o modelo da rede. Serão quatro camadas 
ao invés de inicializar os pesos de forma totalmente aleatória, o glorot_uniform inicializa
com uma distribuição uniforme. Dessa forma, o risco dos gradientes ficarem malucos é minimizado.

In [ ]:

def build_model(width=64, depth=4):
    model = keras.Sequential()
    model.add(layers.Input(shape=(1,)))
    for _ in range(depth-1):
        model.add(layers.Dense(width, activation=tf.nn.tanh, kernel_initializer="glorot_uniform"))
    model.add(layers.Dense(1, activation=None))
    return model

model = build_model()
model.summary()


---
### laplace_residual(model, r) 

Essa função calcula o resíduo da equação de Laplace radial em coordenadas cilíndricas:

$$ \frac{1}{r} \frac{d}{dr} ( r \frac{dV}{dr} ) = 0 $$

que, expandida, fica

$$ \frac{d^2V}{dr^2} + \frac{1}{r}\frac{dV}{dr} =0 $$

Essa função retorna exatamente esse valor para um determinado valor de $r$.

No TensorFlow, o tf.GradientTape é o mecanismo que grava operações para permitir cálculo automático de derivadas. O tape (a "fita" que grava as operações) é descartado assim que você chama .gradient. Ou seja, você só pode calcular um gradiente por vez com aquele tape. Quando se usa persistent = True, o tape não é descartado automaticamente após a primeira chamada. Isso permite calcular várias derivadas (gradientes sucessivos), como no caso de derivadas de ordem superior.


In [ ]:
def laplace_residual(model, r):
    with tf.GradientTape(persistent=True) as tape2:
        tape2.watch(r)
        with tf.GradientTape() as tape1:
            tape1.watch(r)
            V = model(r)
        dV_dr = tape1.gradient(V, r)
    d2V_dr2 = tape2.gradient(dV_dr, r)
    residual = d2V_dr2 + dV_dr/(r+1e-6)
    return residual

# Teste
test_r = tf.constant([[1.5]], dtype=tf.float32)
laplace_residual(model, test_r)


---
### Cálculo da perda

A perda é composta de três componentes:
- o desvio em relação à condição de contorno em R
- o desvio em relação à condição de contorno em R_ext
- O desvio em relação à equação de Laplace

As três perdas são somadas. Também retorna cada componente individualmente para comparação

In [ ]:

def compute_loss():
    # PDE interior
    r_tf = tf.convert_to_tensor(r_interior)
    res = laplace_residual(model, r_tf)
    loss_pde = tf.reduce_mean(tf.square(res))

    # BC em R
    rR = tf.convert_to_tensor(r_bc_R)
    V_R = model(rR)
    loss_bc_R = tf.reduce_mean(tf.square(V_R - V_R_target))

    # BC em R_ext
    rRe = tf.convert_to_tensor(r_bc_Rext)
    V_Rext = model(rRe)
    loss_bc_Rext = tf.reduce_mean(tf.square(V_Rext - V_Rext_target))

    return loss_pde + loss_bc_R + loss_bc_Rext, loss_pde, loss_bc_R, loss_bc_Rext

optimizer = keras.optimizers.Adam(learning_rate=1e-3)


---
### Treinamento da rede

Descrição do código:
1. Loop manual de treinamento (for ep in range(1, epochs+1):) Em vez de chamar model.fit(...), você mesmo controla cada época.
2. Calcula a perda: compute_loss(). Isso é diferente de uma rede tradicional, em que a perda é só, por exemplo, MSE entre predição e dado. Aqui, você precisa combinar várias contribuições físicas + de contorno.
3. Gradientes e atualização dos pesos:
    1. tape.gradient(loss, model.trainable_variables) calcula o gradiente da perda em relação aos pesos.
    2. optimizer.apply_gradients(...) aplica a atualização de pesos.
4. Histórico: Salva cada componente da perda em history para depois plotar curvas de convergência.

#### Por que não usar simplesmente model.fit(...)?

O model.fit do Keras é pensado para aprendizado supervisionado clássico, onde você fornece X (entradas) e y (saídas alvo) e ele minimiza uma função de perda padrão (MSE, cross-entropy, etc.) entre model(X) e y.

No caso das PINNs você não tem "rótulos" tradicionais (y). O que você tem são equações diferenciais e condições de contorno, que você traduz em termos de resíduos.A função de perda é composta manualmente (PDE + contornos).

Logo, precisa implementar o passo de treinamento na mão usando GradientTape.

In [ ]:
def train(epochs, history, verbose = True):
    print_every = epochs/5
    for ep in range(1, epochs+1):
        with tf.GradientTape() as tape:
            loss, lp, lR, lRe = compute_loss()
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
        history["loss"].append(float(loss.numpy()))
        history["pde"].append(float(lp.numpy()))
        history["bc_R"].append(float(lR.numpy()))
        history["bc_Rext"].append(float(lRe.numpy()))
    
        if ep % print_every == 0 and verbose:
            print(f"Epoch {ep:5d} | loss={loss:.3e} | pde={lp:.3e} | bc_R={lR:.3e} | bc_Rext={lRe:.3e}")




In [ ]:
history = {"loss": [], "pde": [], "bc_R": [], "bc_Rext": []}

---
### Treina e mostra resultados evoluindo com treinamento. Comparação com solução analítica

In [ ]:

# Solução analítica
A = (V_ext - V0) / (np.log(R_ext) - np.log(R))
B = V0 - A*np.log(R)

r_plot = np.linspace(R, R_ext, 400).reshape(-1,1).astype(np.float32)
V_true = A*np.log(r_plot) + B

epochs = [10,40,150,200,200]
total = 0

plt.figure(figsize=(10, 8))

for epoch in epochs:
    train(epoch,history,False)
    total = total + epoch
    V_pred = model(r_plot).numpy()
    plt.plot(r_plot, V_pred, label=f"Modelo depois de {total} épocas")

plt.plot(r_plot, V_true, "--", label="Solução analítica")
plt.xlabel("r")
plt.ylabel("V(r)")
plt.title("Potencial radial (Laplace externo)")
plt.legend()
plt.show()

# Curva de perda
plt.figure(figsize=(10, 8))
plt.yscale('log')
plt.plot(history["loss"], label="loss total")
plt.plot(history["pde"], label="loss PDE")
plt.plot(history["bc_R"], label="loss BC r=R")
plt.plot(history["bc_Rext"], label="loss BC r=R_ext")
plt.xlabel("época")
plt.ylabel("loss")
plt.title("Histórico de treinamento")
plt.legend()
plt.show()
